In [1]:
import torch
import pytorch_lightning as pl
import soundfile as sf
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import Wav2Vec2FeatureExtractor, BertTokenizerFast, Wav2Vec2Model, BertModel

In [151]:
from typing import Dict, List, Union
from transformers import BertTokenizerFast, Wav2Vec2FeatureExtractor

class LibriPreprocessor:
  def __init__(self):
    self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    self.extractor = Wav2Vec2FeatureExtractor.from_pretrained('facebook/wav2vec2-base')
  
  
  def speech_file_to_array_fn(self, data):
    speech_array, sampling_rate = sf.read(data["file"])
    data["speech"] = speech_array
    data["sampling_rate"] = sampling_rate
    data["target_text"] = data["text"]
    return data
    
    
  def prepare_dataset(self, data):    
    # check that all files have the correct sampling rate
    assert (
        len(set(data["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {self.extractor.feature_extractor.sampling_rate}."

    data["input_values"] = self.extractor(data["speech"], sampling_rate=data["sampling_rate"][0]).input_values
    
    tokenized_batch = self.tokenizer(data["target_text"], padding='longest', max_length=128, pad_to_max_length=False)
    data['input_ids'] = tokenized_batch['input_ids']
    data['attention_mask_text'] = tokenized_batch['attention_mask']
    data['token_type_ids_text'] = tokenized_batch['token_type_ids']
    
    return data


  def __call__(
    self,
    batch: List[Dict[str, Union[List[int], torch.Tensor]]],
    ) -> Dict[str, torch.Tensor]:
    """
    Collate function to be used when training with PyTorch Lightning.
    Args:
        extractor (:class:`~transformers.Wav2Vec2FeatureExtractor`):
            The processor used for proccessing the data.
        tokenizer (:class:`~transformers.BertTokenizerFast`)
            The tokenizer used for proccessing the data.
        features (:obj:`List[Dict[str, Union[List[int], torch.Tensor]]]`):
            A list of features to be collated.
    Returns:
        :obj:`Dict[str, torch.Tensor]`: A dictionary of tensors containing the collated features.
    """ 
    input_features = [{"input_values": feature["input_values"]} for feature in batch]
    input_sentences = [{"input_ids": feature["input_ids"]} for feature in batch]
    
    speech_batch = self.extractor.pad(
        input_features,
        padding='longest',
        return_tensors="pt",
        )
    text_batch = self.tokenizer.pad(
        input_sentences,
        padding='longest',
        return_tensors='pt'
    )
    
    return speech_batch, text_batch



class LibriSpeechDataset(Dataset):
  def __init__(self, libri_dataset):
    self.libri_dataset = libri_dataset
  
  
  def __len__(self):
    return len(self.libri_dataset)
  
  
  def __getitem__(self, index):
    return self.libri_dataset[index]

In [152]:
libri = load_dataset('patrickvonplaten/librispeech_asr_dummy', 'clean', split='validation')
preprocessor = LibriPreprocessor()
libri_read = libri.map(preprocessor.speech_file_to_array_fn, remove_columns=['chapter_id', 'id', 'speaker_id'])
libri_prepared = libri_read.map(preprocessor.prepare_dataset, batch_size=8, num_proc=4, batched=True)
libri_dataset = LibriSpeechDataset(libri_prepared)
libri_dataloader = DataLoader(libri_dataset, batch_size=4, collate_fn=preprocessor)

Reusing dataset librispeech_asr (C:\Users\marco\.cache\huggingface\datasets\patrickvonplaten___librispeech_asr\clean\2.1.0\f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc)
100%|██████████| 73/73 [00:00<00:00, 572.42ex/s]


In [158]:
for i, batch in enumerate(libri_dataloader):
  speech, text = batch
  print(speech['input_values'].shape)
  print(text['input_ids'].shape)
  if i == 2:
    break

torch.Size([4, 199760])
torch.Size([4, 86])
torch.Size([4, 470400])
torch.Size([4, 86])
torch.Size([4, 292640])
torch.Size([4, 51])


In [85]:
embeddings = wav2vec2(input_values=input_values['input_values'], attention_mask=input_values['attention_mask'])

c:\Users\marco\Google Drive\Projects\.venv\speech-segment-retrieval\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1093: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


In [89]:
embeddings['last_hidden_state'].shape

torch.Size([1, 292, 768])

In [20]:
inputs = tokenizer(libri[0]['text'].lower(), return_tensors='pt', return_special_tokens_mask=True)
print(inputs)

{'input_ids': tensor([[  101, 12525, 27565,  2121,  2003,  1996, 20121,  1997,  1996,  2690,
          4280,  1998,  2057,  2024,  5580,  2000,  6160,  2010,  8036,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'special_tokens_mask': tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])}


In [2]:
from models import ParallelSpeechAndTextModel

In [3]:
model = ParallelSpeechAndTextModel()

Downloading: 100%|██████████| 420M/420M [00:20<00:00, 21.6MB/s] 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: 100%|██████████| 1.56k/1.56k [00:00<00:00, 1.62MB/s]
Downloading: 100%|██████

AttributeError: 'BertModel' object has no attribute 'hidden_size'